# VQE in DFT with PsiEmbed and Qiskit

Here we define the inputs as required by PsiEmbed. Note that we'll follow the logic of `embedding_module/run_open_shell`.

In [1]:
from spade import fill_defaults
import numpy as np

options = {}
options['geometry'] = """
O          0.00000        0.00000        0.11779
H          0.00000        0.75545       -0.47116
H          0.00000       -0.75545       -0.47116
    """
options['basis'] = 'cc-pvdz' # basis set 
options['low_level'] = 'b3lyp' # level of theory of the environment 
options['high_level'] = 'mp2' # level of theory of the embedded system
options['n_active_atoms'] = 1 # number of active atoms (first n atoms in the geometry string)
options['low_level_reference'] = 'rohf'
options['high_level_reference'] = 'rohf'
options['package'] = 'pyscf'

keywords = fill_defaults(options)

# if (keywords['low_level_reference'] == 'rhf' and 
#         keywords['high_level_reference'] == 'rhf'):
#     run_closed_shell(keywords)
# else:
#     run_open_shell(keywords)

The first step is to run a mean field caluclation of the whole system.

The Embed class and its subclasses have a method to do this which also sets the following properties:
    Exchange correlation potentials (v_xc_total if embedding potential is not set, or alpha/beta_v_xc_total)
    

In [4]:
from spade import Embed, PySCFEmbed

embed = PySCFEmbed(keywords)

embed.run_mean_field(v_emb=None)

alpha_occupied_orbitals = embed.alpha_occupied_orbitals
beta_occupied_orbitals = embed.beta_occupied_orbitals

Initialize <pyscf.gto.mole.Mole object at 0x7f528022efd0> in <pyscf.dft.roks.ROKS object at 0x7f528022e3a0>


overwrite output file: output.dat


Next we may want to project to a different basis.

In any case we want to find the orbitals of the active space and environment

In [5]:
    # Whether or not to project occupied orbitals onto another (minimal) basis
    if 'occupied_projection_basis' in keywords:
        op_basis = keywords['occupied_projection_basis']

        alpha_projection_orbitals = embed.basis_projection(
            alpha_occupied_orbitals, op_basis)
        beta_projection_orbitals = embed.basis_projection(
            beta_occupied_orbitals, op_basis)
        n_act_aos = embed.count_active_aos(op_basis)
    else:
        alpha_projection_orbitals = alpha_occupied_orbitals
        beta_projection_orbitals = beta_occupied_orbitals
        n_act_aos = embed.count_active_aos(keywords['basis'])

    ao_overlap = embed.ao_overlap

    # Orbital rotation and partition into subsystems A and B
    alpha_rotation_matrix, alpha_sigma = embed.orbital_rotation(
        alpha_projection_orbitals, n_act_aos, ao_overlap)
    beta_rotation_matrix, beta_sigma = embed.orbital_rotation(
        beta_projection_orbitals, n_act_aos, ao_overlap)

    alpha_n_act_mos, _, beta_n_act_mos, _ = (
        embed.orbital_partition(alpha_sigma, beta_sigma))

    alpha_act_orbitals = (alpha_occupied_orbitals
        @ alpha_rotation_matrix.T[:, :alpha_n_act_mos])
    alpha_env_orbitals = (alpha_occupied_orbitals
        @ alpha_rotation_matrix.T[:, alpha_n_act_mos:])
    beta_act_orbitals = (beta_occupied_orbitals
        @ beta_rotation_matrix.T[:, :beta_n_act_mos])
    beta_env_orbitals = (beta_occupied_orbitals
        @ beta_rotation_matrix.T[:, beta_n_act_mos:])

    alpha_act_density = alpha_act_orbitals @ alpha_act_orbitals.T
    beta_act_density = beta_act_orbitals @ beta_act_orbitals.T
    alpha_env_density = alpha_env_orbitals @ alpha_env_orbitals.T
    beta_env_density = beta_env_orbitals @ beta_env_orbitals.T

Initialize <pyscf.gto.mole.Mole object at 0x7f5280297fa0> in <pyscf.scf.hf.RHF object at 0x7f5280297e50>


We then calclate the cross subsytem terms

In [6]:
# Retrieving the subsytem energy terms and potential matrices
(e_act, e_xc_act, alpha_j_act, beta_j_act, alpha_k_act,
    beta_k_act, alpha_v_xc_act, beta_v_xc_act) = (
    embed.open_shell_subsystem(alpha_act_orbitals, beta_act_orbitals))
    
(e_env, e_xc_env, alpha_j_env, beta_j_env, alpha_k_env,
    beta_k_env, alpha_v_xc_env, beta_v_xc_env) = (
    embed.open_shell_subsystem(alpha_env_orbitals, beta_env_orbitals))

# We need to use the matrix_dot function
matrix_dot = lambda A, B: np.einsum('ij,ij', A, B)

# Computing cross subsystem terms
j_cross = 0.5*(matrix_dot(alpha_j_act, alpha_env_density)
        + matrix_dot(alpha_j_act, beta_env_density)
        + matrix_dot(beta_j_act, alpha_env_density)
        + matrix_dot(beta_j_act, beta_env_density)
        + matrix_dot(alpha_j_env, alpha_act_density)
        + matrix_dot(alpha_j_env, beta_act_density)
        + matrix_dot(beta_j_env, alpha_act_density)
        + matrix_dot(beta_j_env, beta_act_density))
        
k_cross = -0.5*embed.alpha*(matrix_dot(alpha_k_act, alpha_env_density)
        + matrix_dot(beta_k_act, beta_env_density)
        + matrix_dot(alpha_k_env, alpha_act_density)
        + matrix_dot(beta_k_env, beta_act_density))

xc_cross = embed.e_xc_total - e_xc_act - e_xc_env
two_e_cross = j_cross + k_cross + xc_cross

We can now define the projector used to orthogonalise the Molecular and Atomic orbitals. From this we calculate the embedding potential.

In [7]:
    # Defining the embedding potential
    ao_overlap = embed.ao_overlap
    alpha_projector = keywords['level_shift']*(ao_overlap @ alpha_env_density
        @ ao_overlap)
    beta_projector = keywords['level_shift']*(ao_overlap @ beta_env_density
        @ ao_overlap)
        
    alpha_v_emb = (alpha_j_env + beta_j_env - embed.alpha*alpha_k_env
                + alpha_projector + embed.alpha_v_xc_total - alpha_v_xc_act)
    beta_v_emb = (alpha_j_env + beta_j_env - embed.alpha*beta_k_env
                + beta_projector + embed.beta_v_xc_total - beta_v_xc_act)
            

In [11]:
alpha_act_orbitals
alpha

array([[-9.77322452e-01,  2.05528631e-01,  2.55462089e-16],
       [ 9.43799366e-02,  3.06330507e-01, -9.20233620e-15],
       [ 1.27319348e-01,  4.65670155e-01, -1.37110400e-14],
       [ 3.94237547e-15,  1.78060503e-14,  6.36220438e-01],
       [ 1.45486443e-15,  3.48087384e-15,  1.23262117e-15],
       [ 7.17574986e-02,  4.58763476e-01, -1.32571965e-14],
       [ 2.99531699e-15,  1.35686018e-14,  4.88727672e-01],
       [-2.27393653e-16, -1.54440723e-15,  1.53179985e-15],
       [ 5.82462631e-02,  3.24601990e-01, -9.46385489e-15],
       [-1.08018080e-17, -4.76079435e-17,  2.49650354e-16],
       [ 1.84714953e-17,  1.85808235e-16,  1.35167811e-16],
       [-2.64623374e-03, -1.61327153e-02,  4.30660837e-16],
       [-1.18048807e-16, -5.29713376e-16, -1.93364565e-02],
       [-1.43990592e-08,  2.16373012e-04,  3.53124914e-18],
       [ 2.56624200e-03, -1.11998930e-01,  3.11565965e-15],
       [-6.14086766e-03, -3.79794374e-02, -3.09885932e-16],
       [ 1.95987651e-16,  8.83527027e-16

Here, PsiEmbed gives us the option to stop, outputting values for calculation by other means.

To continue, we run the mean field method, but with the embedding potentials as calulated.

In [ ]:

    embed.run_mean_field([alpha_v_emb, beta_v_emb])

    # Overlap between the C_A determinant and the embedded determinant
    embed.determinant_overlap(alpha_act_orbitals, beta_act_orbitals)
    
    # Computing the embedded SCF energy
    alpha_density_emb = (embed.alpha_occupied_orbitals
        @ embed.alpha_occupied_orbitals.T)
    beta_density_emb = (embed.beta_occupied_orbitals
        @ embed.beta_occupied_orbitals.T)
    
    alpha_j_emb = embed.alpha_j
    beta_j_emb = embed.beta_j
    alpha_k_emb = embed.alpha_k
    beta_k_emb = embed.beta_k
    h_core = embed.h_core

    e_act_emb = (matrix_dot(alpha_density_emb + beta_density_emb, h_core)
                + 0.5*matrix_dot(alpha_density_emb + beta_density_emb,
                alpha_j_emb + beta_j_emb)
                - 0.5*(matrix_dot(alpha_density_emb, alpha_k_emb)
                + matrix_dot(beta_density_emb, beta_k_emb)))
                
    correction = (matrix_dot(alpha_v_emb, alpha_density_emb - alpha_act_density)
                + matrix_dot(beta_v_emb, beta_density_emb - beta_act_density))


Initialize <pyscf.gto.mole.Mole object at 0x7fac2a1fa8e0> in <pyscf.scf.rohf.ROHF object at 0x7fac2a1faa60>
/home/miw/.local/lib/python3.8/site-packages/pyscf/gto/mole.py:1045: UserWarning: Function mol.dumps drops attribute _nelectron because it is not JSON-serializable
  warnings.warn(msg)


# This is where we add all the parts up.

e_act_emb : $\epsilon[\gamma^A_{emb}]$
>energy of the embedded region

e_env : $E[\gamma^A]$ 
>energy of the environment

two_e_cross : $g[\gamma^A, \gamma^B]$
>non-additive two electron term

embed.nre
>The Coulomb energy from nuclear repulsion.

correction : $tr[(\gamma^A_{emb} - \gamma^A)(h^{A in B} - h)]$ (or $tr[\gamma^A(h^{A in B} - h)]$ )
> Correction for embedding

In [ ]:


    e_mf_emb = e_act_emb + e_env + two_e_cross + embed.nre + correction

    embed.print_scf(e_act, e_env, two_e_cross, e_act_emb, correction)


82.01074477789119

In [ ]:
from qiskit.aqua.algorithms import VQE, NumPyEigensolver
import matplotlib.pyplot as plt
import numpy as np
from qiskit.chemistry.components.variational_forms import UCCSD
from qiskit.chemistry.components.initial_states import HartreeFock
from qiskit.circuit.library import EfficientSU2
from qiskit.aqua.components.optimizers import COBYLA, SPSA, SLSQP
from qiskit.aqua.operators import Z2Symmetries
from qiskit import IBMQ, BasicAer, Aer
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.chemistry import FermionicOperator
from qiskit.aqua import QuantumInstance
from qiskit.ignis.mitigation.measurement import CompleteMeasFitter
from qiskit.providers.aer.noise import NoiseModel

from embedding_methods import Embed

from qiskit import Aer
from qiskit.aqua import QuantumInstance, aqua_globals
from qiskit.aqua.operators import OperatorBase
from qiskit.algorithms import VQE, NumPyMinimumEigensolver
from qiskit.algorithms.optimizers import SPSA
from qiskit.circuit.library import TwoLocal
from qiskit.aqua.operators import I, X, Z
from qiskit_nature.circuit.library import UCCSD
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper, ParityMapper
from qiskit.chemistry import FermionicOperator
from qiskit_nature.drivers import PySCFDriver
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem


/home/miw/miniconda3/lib//python3.8/site-packages/qiskit/aqua/__init__.py:86: DeprecationWarning: The package qiskit.aqua is deprecated. It was moved/refactored to qiskit-terra For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua', 'qiskit-terra')
/home/miw/miniconda3/lib//python3.8/site-packages/qiskit/chemistry/__init__.py:170: DeprecationWarning: The package qiskit.chemistry is deprecated. It was moved/refactored to qiskit_nature (pip install qiskit-nature). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('chemistry', 'qiskit_nature', 'qiskit-nature')


In [ ]:
def get_qubit_op(dist):
    driver = PySCFDriver(atom=options["geometry"], 
        unit=UnitsType.ANGSTROM, 
        charge=0, 
        spin=0, 
        basis=options["basis"],
        )
        
    molecule = driver.run()
    freeze_list = [0]
    remove_list = [-3, -2]
    repulsion_energy = molecule.nuclear_repulsion_energy
    num_particles = molecule.num_alpha + molecule.num_beta
    num_spin_orbitals = molecule.num_orbitals * 2
    remove_list = [x % molecule.num_orbitals for x in remove_list]
    freeze_list = [x % molecule.num_orbitals for x in freeze_list]
    remove_list = [x - len(freeze_list) for x in remove_list]
    remove_list += [x + molecule.num_orbitals - len(freeze_list)  for x in remove_list]
    freeze_list += [x + molecule.num_orbitals for x in freeze_list]
    ferOp = FermionicOperator(h1=molecule.one_body_integrals, h2=molecule.two_body_integrals)
    ferOp, energy_shift = ferOp.fermion_mode_freezing(freeze_list)
    num_spin_orbitals -= len(freeze_list)
    num_particles -= len(freeze_list)
    ferOp = ferOp.fermion_mode_elimination(remove_list)
    num_spin_orbitals -= len(remove_list)
    qubitOp = ferOp.mapping(map_type='parity', threshold=0.00000001)
    qubitOp = Z2Symmetries.two_qubit_reduction(qubitOp, num_particles)
    shift = energy_shift + repulsion_energy
    return qubitOp, num_particles, num_spin_orbi